## **Load libraries and data**

In [22]:
#!pip install --upgrade pythainlp
!python -m pip install pyLDAvis

Defaulting to user installation because normal site-packages is not writeable


In [23]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [24]:
df = pd.read_csv('https://github.com/Surawich2021/Surawich-6310422083/raw/main/Homework11%20-%20Voice%20of%20Customer/Wongnai%20Reviews%20-%20Small.csv')

In [25]:
df.tail()

,Review ID,Review
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...


## **Tokenize Words**

In [26]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n', 'ร้าน', '(', ')' , '           ']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [27]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [28]:
df.tail()

,Review ID,Review,Review_tokenized
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...,"ค่ำ,เพื่อน,ส้มตำ,หมู,เฮา,ลงมา,กิน,ส้มตำ,ออฟฟิศ..."
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...,"สะอาด,ดี,ตกแต่ง,สวยงาม,ที่จอดรถ,ราคา,เมนู,เทีย..."
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...,"เช้า,รีบ,วิ่ง,เข่า,ห้องเรียน,แทบ,ต้องหา,ของกิน..."
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...,"ร้านกาแฟ,Happy,Mango,อาทิตย์,นัด,เพื่อน,นั่ง,ค..."
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...,"ทรู,คอ,ฟ,ฟี่,สาขา,ซีคอน,ศูนย์,บริการ,ทรู,ชั้น,..."


## **Create Dictionary**

In [29]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [30]:
print(dictionary.token2id.keys())

dict_keys(['20', 'Macchiato', 'กาแฟ', 'กาแฟร้อน', 'กิน', 'คน', 'ครึ่ง', 'ความคิด', 'ชอบ', 'ดื่ม', 'ตอน', 'ทาน', 'นึง', 'บาท', 'ปริมาณ', 'มีความสุข', 'ลา', 'สั่ง', 'หนัก', 'หลังจากนี้', 'เข้าไป', 'เจอ', 'เดิน', 'เดิม', 'เป็นประจำ', 'เฟล', 'แก้ว', 'โบราณ', '-', 'Art', 'Coffee', 'Kasetsart', '^^', 'of', 'คาราเมล', 'คิ', 'ชา', 'ซ', 'ซี', 'ดี', 'นั่ง', 'น่ารัก', 'บรรยากาศ', 'พนักงาน', 'พูดจา', 'มัค', 'รสชาติ', 'ราคา', 'ร้านกาแฟ', 'ลอน', 'ลาเต้', 'ลูกค้า', 'อัธยาศัย', 'อัสสัม', 'อา', 'อิตาเลียน', 'อุดหนุน', 'เพรส', 'เมนู', 'เยี่ยม', 'เอ๊กซ์', 'แพง', 'โซดา', 'โซ่', 'โต้', 'ไอ', 'ไอซ์', '555', 'กก', 'กรอบ', 'กระ', 'กระทะ', 'กร๊อบ', 'กลิ่น', 'กวง', 'กะ', 'กินน้ำ', 'กุ้ง', 'ก้น', 'ข้าวผัด', 'ข้าวเหนียว', 'ความสนใจ', 'คอ', 'คาว', 'งั้น', 'งี้', 'จริ๊ง', 'จัดจ้าน', 'จาน', 'จี๊ดจ๊าด', 'จ้น', 'จ้ิม', 'จ๊อบ', 'ฉ่า', 'ชิม', 'ชุ่มคอ', 'ดาว', 'ตบท้าย', 'ตัก', 'ถูกใจ', 'ทอด', 'ทะเล', 'ทีหลัง', 'นานา', 'นุ่ม', 'น้ำ', 'น้ำปลา', 'บรรยาย', 'ปลา', 'ปลาหมึก', 'ปาก', 'ปู', 'ปูม้า', 'ผสม', 'ผัด', 'พง', 'พริก', '

In [31]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

## **Topic Modeling**

In [32]:
num_topics = 7
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

Wall time: 6.1 s


In [34]:
#pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)
pyLDAvis.gensim_models.prepare(model, gensim_corpus, dictionary, mds='mmds') 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.014271  0.018390       1        1  35.332736
0     -0.027839 -0.138430       2        1  13.876727
4     -0.116345 -0.052294       3        1  12.753508
2     -0.098691  0.097890       4        1  11.945484
6      0.092472 -0.086464       5        1  11.793495
5      0.105084  0.019077       6        1   8.753571
1      0.059589  0.141831       7        1   5.544480, topic_info=         Term        Freq       Total Category  logprob  loglift
914    ไข่มุก   93.000000   93.000000  Default  30.0000  30.0000
36         ชา  227.000000  227.000000  Default  29.0000  29.0000
438        นม  132.000000  132.000000  Default  28.0000  28.0000
663         :  109.000000  109.000000  Default  27.0000  27.0000
68         กก   77.000000   77.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
39         ดี    7.480033  323.153875   Topic7  -5.5646  -0.8735
753      ซื้อ    5.666003   61.150856   Topic7  -5.8424   0.5135
42   บรรยากาศ    6.188780  132.757682   Topic7  -5.7541  -0.1734
46     รสชาติ    6.763672  363.056051   Topic7  -5.6653  -1.0906
40       นั่ง    5.876775  177.981571   Topic7  -5.8059  -0.5183

[564 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3691      3  0.802738         
318       1  0.576579        "
318       2  0.076877        "
318       3  0.166567        "
318       5  0.038439        "
...     ...       ...      ...
1534      3  0.727309        ”
1534      5  0.121218        ”
1534      6  0.121218        ”
4651      1  0.867647        •
4693      6  0.840738       ••

[1141 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 5, 3, 7, 6, 2])

In [35]:
model.show_topic(3)

[('อร่อย', 0.014263497),
 ('รสชาติ', 0.01319392),
 ('ดี', 0.012791566),
 ('ทาน', 0.012574317),
 ('อาหาร', 0.009788351),
 ('สั่ง', 0.0097217),
 ('กาแฟ', 0.008836086),
 ('กิน', 0.008748116),
 ('ชอบ', 0.007645375),
 ('เมนู', 0.0074249343)]

In [36]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [38]:
#df[['Restaurant', 'Review', 'topics']]
df[['Review', 'topics']]

,Review,topics
0,เป็นคนที่ชอบทาน Macchiato เป็นประจำ มีวันนึงเด...,3
1,Art of Coffee Kasetsart เป็นร้านกาแฟรสชาติเยี่...,3
2,กวงทะเลเผา อาหารทะเลเค้าสดจริงๆเนื้อปูหวานไม่ค...,2
3,วันนี้มีโอกาสตื่นเช้าครับเลยถึงโอกาสออกมาหาอะไ...,3
4,ชอบมาทานร้านนี้ถ้าอยากกินอาหารเวียดนามใกล้บ้าน...,3
...,...,...
295,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...,6
296,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...,6
297,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...,3
298,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...,1
